In [ ]:
import cv2
import os
import numpy as np
import tensorflow as tf
from tensorflow.python.keras.utils import np_utils
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.python.keras.constraints import maxnorm
from tensorflow.keras.optimizers import SGD
from tensorflow.python.keras.layers.convolutional import Conv2D
from tensorflow.python.keras.layers.convolutional import MaxPooling2D

In [ ]:
from tensorflow.python.keras.models import model_from_json
from tensorflow.python.keras.models import load_model
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [ ]:
people = [people for people in os.listdir("output_image/")]
print(people)
num_classes = 3
img_data_list = []
labels = []
valid_images = [".jpg"]

['.ipynb_checkpoints', 'hillary', 'jokowi']


In [ ]:
for index, person in enumerate(people):
  print(index)
  dir_path = 'output_image/' + person
  print(dir_path)
  for img_path in os.listdir(dir_path):
    name, ext = os.path.splitext(img_path)
    if ext.lower() not in valid_images:
        continue
    img_data = cv2.imread(dir_path + '/' + img_path)
    # convert image to gray
    img_data=cv2.cvtColor(img_data, cv2.COLOR_BGR2GRAY)
    img_data_list.append(img_data)
    labels.append(index)
img_data = np.array(img_data_list)
img_data = img_data.astype('float32')

labels = np.array(labels ,dtype='int64')
# scale down(so easy to work with)
img_data /= 255.0
img_data= np.expand_dims(img_data, axis=3)
# convert class labels to on-hot encoding
Y = np_utils.to_categorical(labels, num_classes)
#Shuffle the dataset
x,y = shuffle(img_data,Y, random_state=2)
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2)

0
output_image/.ipynb_checkpoints
1
output_image/hillary
2
output_image/jokowi


In [ ]:
input_shape=img_data[0].shape
print(input_shape)
model= tf.keras.models.Sequential([ 
  tf.keras.layers.Conv2D(32, (3, 3), input_shape=input_shape, padding='same', activation='relu',
                 kernel_constraint=maxnorm(3)),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same', kernel_constraint=maxnorm(3)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu', kernel_constraint=maxnorm(3)),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(num_classes, activation='softmax')
])


(64, 64, 1)


In [ ]:
epochs = 100
learning_rate = 0.01
sgd = SGD(learning_rate=learning_rate, momentum=0.9, nesterov=False)
model.compile(loss='categorical_crossentropy',
                    optimizer= sgd,
                    metrics=['accuracy'])
print(model.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 64, 64, 32)        320       
                                                                 
 dropout (Dropout)           (None, 64, 64, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 32)        9248      
                                                                 
 flatten (Flatten)           (None, 131072)            0         
                                                                 
 dense (Dense)               (None, 512)               67109376  
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 3)                 1

In [ ]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=32)

Epoch 1/100
1/1 [==============================] - 2s 2s/step - loss: 1.1064 - accuracy: 0.2667 - val_loss: 0.8732 - val_accuracy: 0.5000
Epoch 2/100
1/1 [==============================] - 1s 1s/step - loss: 0.9156 - accuracy: 0.6667 - val_loss: 0.7219 - val_accuracy: 1.0000
Epoch 3/100
1/1 [==============================] - 1s 1s/step - loss: 0.7840 - accuracy: 0.5333 - val_loss: 0.6492 - val_accuracy: 0.7500
Epoch 4/100
1/1 [==============================] - 1s 1s/step - loss: 0.6169 - accuracy: 0.8000 - val_loss: 0.5935 - val_accuracy: 0.7500
Epoch 5/100
1/1 [==============================] - 1s 1s/step - loss: 0.6388 - accuracy: 0.6667 - val_loss: 0.5435 - val_accuracy: 1.0000
Epoch 6/100
1/1 [==============================] - 1s 1s/step - loss: 0.6420 - accuracy: 0.6667 - val_loss: 0.8134 - val_accuracy: 0.5000
Epoch 7/100
1/1 [==============================] - 1s 1s/step - loss: 0.8164 - accuracy: 0.6000 - val_loss: 1.0773 - val_accuracy: 0.5000
Epoch 8/100
1/1 [=================

In [ ]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 75.00%


In [ ]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

In [ ]:
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


TESTING

In [ ]:
from imutils import face_utils
from imutils.face_utils import FaceAligner
import imutils
import time
import cv2
import dlib
import os
import numpy as np

In [ ]:
from tensorflow.python.keras.models import model_from_json
from tensorflow.python.keras.models import load_model

In [ ]:
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

In [ ]:
loaded_model.load_weights("model.h5")
print("Loaded model from disk")
print("[INFO] loading facial landmark predictor...")
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
fa = FaceAligner(predictor, desiredFaceWidth=64)

Loaded model from disk
[INFO] loading facial landmark predictor...


RuntimeError: ignored